In [32]:
# !pip install xlsxwriter
import pandas as pd
import numpy as np
import glob
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.options.display.max_rows = 15
pd.set_option('display.max_columns', None)


In [33]:
def autosize_excel_columns(worksheet, df):
  autosize_excel_columns_df(worksheet, df.index.to_frame())
  autosize_excel_columns_df(worksheet, df, offset=df.index.nlevels)

def autosize_excel_columns_df(worksheet, df, offset=0):
  for idx, col in enumerate(df):
    series = df[col]
    max_len = max((
      series.astype(str).map(len).max(),
      len(str(series.name))
    )) + 1
    worksheet.set_column(idx+offset, idx+offset, max_len)

In [34]:
def create_report(path):
    xlsx_list = sorted(glob.glob(f'{path}*.xlsx'))
    output = pd.DataFrame()
    for name in xlsx_list:
        df = pd.read_excel(name, sheet_name = 'сводная таблица',
                           index_col = [0, 1],
                           usecols = [0, 1, 2, 3, 4])
        df.drop(index='Все', inplace=True)
        sums = df.groupby(level = 0).sum().drop(columns='Процент неликвидов')
        df.drop(columns = ['Всего новинок', 'Всего неликвидов'], inplace=True)
        sums['ИТОГ % НЕЛИКВИДОВ'] = sums['Всего неликвидов'] / sums['Всего новинок'] * 100
        sums.rename(columns={'Всего новинок':'ИТОГ ШТ. НОВИНОК', 'Всего неликвидов':'ИТОГ ШТ. НЕЛИКВИДОВ'}, inplace=True)
        sums = sums.stack().to_frame(name='Процент неликвидов')
        sums['dummy'] = 1
        period = pd.concat([df, sums])
        period.rename(columns={'Процент неликвидов':name[-15:-8]}, inplace=True)
        output = pd.concat([output, period], axis=1)
    output['sorter'] = output['dummy'].sum(axis=1)
    output = output.sort_values(by=['Редакция', 'sorter']).drop(columns=['dummy', 'sorter'])
    return output

In [35]:
def write_output(output, name='output.xlsx'):
# Write pivot table to file
    writer = pd.ExcelWriter(name , engine='xlsxwriter') 
    output.to_excel(writer, sheet_name='сводная таблица', na_rep='')
    worksheet = writer.sheets['сводная таблица']
    autosize_excel_columns(worksheet, output)
# Write graphs to file
    writer.close()
    print('done')

In [50]:
output = create_report('input/ЭКСМО/')
# Get graph data
output.reset_index(inplace=True)
graph1_data = output.loc[lambda df: df['Подразделение'].isin(['ИТОГ % НЕЛИКВИДОВ']), :]
graph1_data = graph1_data.loc[lambda df: df['Редакция'].isin(['100 - Редакция "Комильфо"',
                                                '101 - Редакция № 1',
                                                '102 - Редакция № 2',
                                                '120 - Редакция № 5']), :]
graph1_data

,Редакция,Подразделение,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10
3,"100 - Редакция ""Комильфо""",ИТОГ % НЕЛИКВИДОВ,23.783784,14.728682,13.028169,16.666667,15.053763,15.384615,16.838488,21.107266,19.536424,17.000000,15.017065,14.878893
21,101 - Редакция № 1,ИТОГ % НЕЛИКВИДОВ,12.500000,11.111111,9.087719,10.222851,12.270804,15.005284,12.008578,13.179396,14.081996,17.433853,19.579405,23.424408
48,102 - Редакция № 2,ИТОГ % НЕЛИКВИДОВ,20.958084,16.256158,16.624896,20.909091,24.152192,27.162490,23.393740,26.109215,27.512777,31.921824,32.801277,33.044847
80,120 - Редакция № 5,ИТОГ % НЕЛИКВИДОВ,18.953145,16.330039,15.627669,17.529711,17.578288,18.819188,19.585921,23.188406,25.917431,28.429119,28.391768,27.474207


In [38]:
write_output(output)

done


In [ ]:
# Get the xlsxwriter workbook and worksheet objects.
workbook = writer.book
worksheet = writer.sheets['sheetName']
chart1 = workbook.add_chart({'type': 'line'})

# Fill the chart from the dataframe data
(max_row, max_col) = df.shape
for row in range(1, max_row + 1):
    chart1.add_series({
        'values': ['sheetName', row, 1, row, max_col-1],
        'name': ['sheetName', row, 0, row, 0],
        'line': {'width': 1},
        'smooth': True,
    })



# Insert chart with params
worksheet.insert_chart(max_row + 1, 0, chart1)

